# Multi-computation with field change

**Champ_parametrique** field for chaining different scenarios in a single *datafile*. \
One stationary then one transitory computation are run and compared to two computations, the second one reading the save results of the first one. \
Each computation # is run into a **dedicated directory** named calcul# with **time reset to 0**.

Validation made by P. LEDAC with TRUST 1.9.4.

In [ ]:
from trustutils import run 
import os
jdd="cp"
cases=[
    (jdd, "champ_parametrique", "Champ_Uniforme 1 0.", "champ_parametrique { fichier champ.txt }"),
    ("calcul1", "stationary",   "Champ_Uniforme 1 0.", "Champ_Fonc_MED { use_existing_domain file puissance.med domain dom field TEMPERATURE_ELEM_dom loc elem last_time }"),
    ("calcul2", "transient",    "Champ_fonc_reprise binaire ../calcul1/calcul1_pb.sauv pb temperature last_time", "Champ_Fonc_MED_Table_Temps { \
        table_temps { 5 \
                      0.0 1.0 2.0 3.0 4.0 \
                      1.0 0.9 0.6 0.3 0.2 } \
        use_existing_domain file puissance.med domain dom field TEMPERATURE_ELEM_dom loc elem time 0. } ")
]
#    
run.reset()
run.initCaseSuite()
run.executeCommand("echo { > champ.txt")
separateur=""
# Create test case:
for case,label,init,champ in cases:
    if label=="champ_parametrique":
       rep="."
    else:
       rep=case
       # Build champ.txt file containing the list of fields (stationary then transient):
       run.executeCommand("echo -e %s %s >> champ.txt" % (separateur, champ), verbose=False)
       separateur=","
    run.executeCommand("rep=%s;case=%s;mkdir -p $rep;cp base.data $rep/$case.data;cp puissance.med $rep;cp verifie $rep/post_run" % (rep,case), verbose=False)
    cas = run.addCase(rep ,"%s.data" % case) 
    cas.substitute("_init_",init)
    cas.substitute("_champ_",champ)

run.executeCommand("echo } >> champ.txt")
run.printCases()
run.runCases(preventConcurrent=True)

## Check results
### The computation (single or multi) should match

In [ ]:
from trustutils import plot 
import numpy as np
import matplotlib.pyplot as plt 
jdd="cp"
for case,label,init,champ in cases:
    if label!="champ_parametrique":
        a=plot.Graph("Probes for %s" % label,size=[15,8])
        a.addPoint("%s/%s_T.son"  % (case,case), marker='-', label="Temperature (Single computation)")
        a.addPoint("%s/%s_T.son"  % (case,jdd) , marker='o', markevery=10, label="Temperature (Multi-computation)")
        a.label("Time","Temperature")